In [1]:
import pandas as pd # Pour manipuler des dataframes
import numpy as np # Pour manipuler des arrays
import matplotlib.pyplot as plt # Pour afficher des graphiques
import seaborn as sns # Pour améliorer l'affichage des graphiques
import duckdb # Pour manipuler des bases de données
from datetime import datetime # Pour manipuler des dates

# 1. Créations

## 1.1 LECTURE + INSERTION BDD

In [2]:
# Connexion ou création de la base de données
con = duckdb.connect("my_database.duckdb")

# Liste des fichiers Parquet à insérer
files = [
    'data/2019-Oct.parquet',
    'data/2019-Nov.parquet',
    'data/2019-Dec.parquet',
    'data/2020-Jan.parquet',
    'data/2020-Feb.parquet',
    'data/2020-Mar.parquet',
    'data/2020-Apr.parquet'
]

# Créer la table si elle n'existe pas
con.execute("""
    CREATE TABLE IF NOT EXISTS data AS SELECT * FROM parquet_scan('{}') WHERE 1=0
""".format(files[0]))

# Insérer les fichiers un par un
for file in files:
    print(f"Inserting data from {file}, starting time : {datetime.now()}")
    con.execute(f"""
        INSERT INTO data SELECT * FROM parquet_scan('{file}');
    """)

print("Data successfully inserted!")

# Vérification (optionnel)
sample_data = con.execute("SELECT * FROM data LIMIT 5").fetchall()
print("Sample Data:", sample_data)

# Fermer la connexion
con.close()

Inserting data from data/2019-Oct.parquet, starting time : 2024-12-04 12:30:59.282672


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Inserting data from data/2019-Nov.parquet, starting time : 2024-12-04 12:31:13.120541


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Inserting data from data/2019-Dec.parquet, starting time : 2024-12-04 12:31:41.595273


...

Inserting data from data/2020-Mar.parquet, starting time : 2024-12-04 12:34:17.232461


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Inserting data from data/2020-Apr.parquet, starting time : 2024-12-04 12:34:46.828695


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Data successfully inserted!
Sample Data: [('2019-10-01 00:00:00 UTC', 'view', 44600062, 2103807459595387724, None, 'shiseido', 35.79, 541312140, '72d76fde-8bb3-4e00-8c23-a032dfed738c'), ('2019-10-01 00:00:00 UTC', 'view', 3900821, 2053013552326770905, 'appliances.environment.water_heater', 'aqua', 33.2, 554748717, '9333dfbd-b87a-4708-9857-6336556b0fcc'), ('2019-10-01 00:00:01 UTC', 'view', 17200506, 2053013559792632471, 'furniture.living_room.sofa', None, 543.1, 519107250, '566511c2-e2e3-422b-b695-cf8e6e792ca8'), ('2019-10-01 00:00:01 UTC', 'view', 1307067, 2053013558920217191, 'computers.notebook', 'lenovo', 251.74, 550050854, '7c90fc70-0e80-4590-96f3-13c02c18c713'), ('2019-10-01 00:00:04 UTC', 'view', 1004237, 2053013555631882655, 'electronics.smartphone', 'apple', 1081.98, 535871217, 'c6bd7419-2748-4c56-95b4-8cec9ff8b80d')]


## 1.2 NETTOYAGE

Nettoyage selon plusieurs règles:
- Suppression de toutes les lignes où brand = 'None' ou NULL, (50 000 000 lignes)

In [3]:
import duckdb

# Connexion à la base de données
con = duckdb.connect("my_database.duckdb")

# Comptage des lignes initiales dans la table 'data'
initial_count = con.execute("SELECT COUNT(*) FROM data").fetchall()[0][0]
print(f"Initial number of rows: {initial_count}")

# Créer une table temporaire sans les lignes à supprimer (celles qui ont 'brand' NULL ou 'None')
con.execute("""
    DELETE FROM data
    WHERE (brand IS NULL OR brand = 'None');
""")

# Comptage des lignes restantes dans la table temporaire
final_count = con.execute("SELECT COUNT(*) FROM data").fetchall()[0][0]

# Calcul du nombre de lignes supprimées
deleted_count = initial_count - final_count

# Afficher le nombre de lignes supprimées
print(f"Number of rows deleted: {deleted_count}")

# Fermer la connexion
con.close()


Initial number of rows: 375484362


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Number of rows deleted: 50752718


## 1.3 Création de la table User

In [ ]:
import duckdb

# Connexion à la base de données
con = duckdb.connect("my_database.duckdb")

# Supprimer la table 'Utilisateur' si elle existe déjà
con.execute("DROP TABLE IF EXISTS Utilisateur")

# Créer une table 'Utilisateur' avec les informations agrégées
con.execute("""
    CREATE TABLE Utilisateur AS
    SELECT
        user_id AS identifiant_utilisateur,
        
        -- Calcul du total dépensé par l'utilisateur (en filtrant par les événements de type 'purchase')
        SUM(CASE WHEN price IS NOT NULL AND event_type = 'purchase' THEN price ELSE 0 END) AS total_depense,
        
        -- Calcul du nombre total d'achats effectués (seulement les événements de type 'purchase')
        COUNT(DISTINCT CASE WHEN event_type = 'purchase' AND price > 0 THEN user_session END) AS nombre_total_achats,
        
        -- Calcul du nombre total de vues effectuées (par session)
        COUNT(DISTINCT CASE WHEN event_type = 'view' AND price IS NULL THEN user_session END) AS nombre_total_vues,
        
        -- Calcul du nombre de catégories uniques vues
        COUNT(DISTINCT category_code) AS categories_uniques_vues,
        
        -- Calcul du nombre de produits uniques vus
        COUNT(DISTINCT product_id) AS produits_uniques_vus,
        
        -- Calcul du nombre total de sessions de l'utilisateur
        COUNT(DISTINCT user_session) AS total_sessions,
        
        -- Calcul de l'achat moyen (seulement pour les événements de type 'purchase')
        AVG(CASE WHEN event_type = 'purchase' AND price > 0 THEN price ELSE NULL END) AS achat_moyen,
        
        -- Vérifier si l'utilisateur a acheté un ebook (dans category_code)
        MAX(CASE WHEN category_code = 'computers.ebooks' AND event_type = 'purchase' THEN 1 ELSE 0 END) AS a_achete_des_ebooks
        
    FROM
        data
    GROUP BY
        user_id
    LIMIT 10;  -- Limite de 10 utilisateurs pour l'exemple, à enlever si on veut tous les utilisateurs
""")

# Affichage du nombre de lignes insérées dans la nouvelle table 'Utilisateur'
nombre_lignes_nouvelle_table = con.execute("SELECT COUNT(*) FROM Utilisateur").fetchall()[0][0]
print(f"Nombre de lignes insérées dans la table Utilisateur : {nombre_lignes_nouvelle_table}")

# Fermer la connexion
con.close()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
import duckdb

# Connexion à la base de données
con = duckdb.connect("my_database.duckdb")

# Supprimer la table 'Utilisateur' si elle existe déjà
con.execute("DROP TABLE IF EXISTS Utilisateur")

# Créer la table 'Utilisateur' avec les résultats agrégés
con.execute("""
    CREATE TABLE Utilisateur AS
    SELECT
        d.user_id AS identifiant_utilisateur,

        -- Calcul du total dépensé par l'utilisateur (en filtrant par les événements de type 'purchase')
        SUM(CASE WHEN d.price IS NOT NULL AND d.event_type = 'purchase' THEN d.price ELSE 0 END) AS total_depense,

        -- Calcul du nombre total d'achats effectués (seulement les événements de type 'purchase')
        COUNT(DISTINCT CASE WHEN d.event_type = 'purchase' AND d.price > 0 THEN d.user_session END) AS nombre_total_achats,

        -- Calcul du nombre de catégories uniques vues
        COUNT(DISTINCT d.category_code) AS categories_uniques_vues,

        -- Calcul du nombre de produits uniques vus
        COUNT(DISTINCT d.product_id) AS produits_uniques_vus,

        -- Calcul du nombre total de sessions de l'utilisateur
        COUNT(DISTINCT d.user_session) AS total_sessions,

        -- Calcul de l'achat moyen (seulement pour les événements de type 'purchase')
        COALESCE(AVG(CASE WHEN d.event_type = 'purchase' AND d.price > 0 THEN d.price ELSE NULL END), 0) AS achat_moyen,

        -- Vérifier si l'utilisateur a acheté un ebook (dans category_code)
        MAX(CASE WHEN d.category_code = 'computers.ebooks' AND d.event_type = 'purchase' THEN 1 ELSE 0 END) AS a_achete_des_ebooks

    FROM
        data d
    GROUP BY
        d.user_id;
""")

# Affichage du nombre de lignes insérées dans la table 'Utilisateur'
nombre_lignes_nouvelle_table = con.execute("SELECT COUNT(*) FROM Utilisateur").fetchall()[0][0]
print(f"Nombre de lignes insérées dans la table Utilisateur : {nombre_lignes_nouvelle_table}")

# Fermer la connexion
con.close()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Nombre de lignes insérées dans la table Utilisateur : 13984736


## 1.4 Ajout de la colonne achat ebook amazon

In [8]:
import duckdb

# Connexion à la base de données
con = duckdb.connect("my_database.duckdb")

# Ajouter la colonne 'a_achete_des_ebooks_amazon' à la table 'Utilisateur'
con.execute("""
    ALTER TABLE Utilisateur ADD COLUMN IF NOT EXISTS a_achete_des_ebooks_amazon INTEGER DEFAULT 0;
""")

# Mise à jour de la colonne 'a_achete_des_ebooks_amazon' pour les utilisateurs ayant acheté un ebook de la marque Amazon
con.execute("""
    UPDATE Utilisateur u
    SET a_achete_des_ebooks_amazon = (
        SELECT MAX(CASE WHEN d.category_code = 'computers.ebooks' AND d.event_type = 'purchase' AND d.brand = 'amazon' THEN 1 ELSE 0 END)
        FROM data d
        WHERE d.user_id = u.identifiant_utilisateur
    );
""")

# Affichage du nombre d'utilisateurs ayant acheté des ebooks Amazon
nombre_lignes_mise_a_jour_amazon = con.execute("SELECT COUNT(*) FROM Utilisateur WHERE a_achete_des_ebooks_amazon = 1").fetchall()[0][0]
print(f"Nombre d'utilisateurs ayant acheté des ebooks Amazon : {nombre_lignes_mise_a_jour_amazon}")

# Fermer la connexion
con.close()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Nombre d'utilisateurs ayant acheté des ebooks Amazon : 142


## 1.5 Création de la table Produit

In [16]:
import duckdb

# Connexion à la base de données
con = duckdb.connect("my_database.duckdb")

# Supprimer la table 'Produit' si elle existe déjà
con.execute("DROP TABLE IF EXISTS Produit;")

# Créer la table 'Produit' avec les informations agrégées
con.execute("""
    CREATE TABLE IF NOT EXISTS Produit AS
    SELECT
        brand AS marque,
        COUNT(DISTINCT product_id) AS nombre_vente,
        ANY_VALUE(price) AS prix,
        category_code AS code_categorie
    FROM
        data
    WHERE
        event_type = 'purchase' AND price > 0
    GROUP BY
        brand, category_code;
""")

# Affichage du nombre de lignes insérées dans la nouvelle table 'Produit'
nombre_lignes_nouvelle_table = con.execute("SELECT COUNT(*) FROM Produit").fetchall()[0][0]
print(f"Nombre de lignes insérées dans la table Produit : {nombre_lignes_nouvelle_table}")

# Fermer la connexion
con.close()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Nombre de lignes insérées dans la table Produit : 14304


# 2. AUTRE

## QUERYS

In [31]:
# Connexion à DuckDB
con = duckdb.connect("my_database.duckdb")

# Charger le fichier Parquet dans un DataFrame Pandas
df = con.execute("SELECT event_time FROM data where brand = 'amazon' category_code = 'computeurs.ebooks'").fetchdf()

# Afficher les 10 premiers utilisateurs
print(df)

# Fermer la connexion
con.close()

ParserException: Parser Error: syntax error at or near "category_code"